In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
print(y)

tf.Tensor(
[[ 0.3101221  -0.17016293  0.35809374  0.11645769]
 [ 0.3101221  -0.17016293  0.35809374  0.11645769]
 [ 0.3101221  -0.17016293  0.35809374  0.11645769]], shape=(3, 4), dtype=float32)


# Another Way

In [3]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))
print(y)

tf.Tensor(
[[-0.01424892  0.01512515  0.03039482  0.04665321]
 [-0.01424892  0.01512515  0.03039482  0.04665321]
 [-0.01424892  0.01512515  0.03039482  0.04665321]], shape=(3, 4), dtype=float32)


# Another Way

In [4]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

In [5]:
model.pop()
print(len(model.layers))  # 2

2


# Input Shape

In [6]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
#model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
#print(model.weights)
#print("Number of weights after calling the model:", len(model.weights))  # 6
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (1, 2)                    10        
                                                                 
 dense_4 (Dense)             (1, 3)                    9         
                                                                 
 dense_5 (Dense)             (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [7]:
for layer in model.layers: 
    print(layer.get_weights())

[array([[-0.01691175, -0.6492183 ],
       [ 0.9247649 , -0.77884483],
       [ 0.8423867 , -0.5356815 ],
       [-0.6365659 ,  0.8886831 ]], dtype=float32), array([0., 0.], dtype=float32)]
[array([[ 0.13885546, -0.21035904,  0.630739  ],
       [-0.25046605,  0.05652523,  0.48193538]], dtype=float32), array([0., 0., 0.], dtype=float32)]
[array([[ 0.7999208 , -0.7419277 , -0.2075671 ,  0.43560612],
       [-0.8832874 , -0.17007542,  0.8647039 ,  0.02798933],
       [-0.5309597 , -0.24200493,  0.80801463, -0.87953687]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32)]


In [8]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [9]:
for layer in model.layers: 
    print(layer.get_weights())

[array([[ 0.8860457 ,  0.40276456],
       [ 0.55605054, -0.03310251],
       [-0.8150332 , -0.38931704],
       [-0.86710143,  0.40350103]], dtype=float32), array([0., 0.], dtype=float32)]


# Add() + Summary()

In [10]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                            

# Feature Extraction With Sequential

In [15]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [19]:
print([print(i.shape) for i in features])

(1, 123, 123, 32)
(1, 121, 121, 32)
(1, 119, 119, 32)
[None, None, None]


# Transfer Learning
## One way

In [23]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
#model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
    print(layer)
    #layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
#model.compile(...)
#model.fit(...)

# Another Way

In [ ]:
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)